# BIZ 프로젝트 : 부실기업 예측 모형에 관한 연구

## Step3 : 모델링

할거 : 21,22(2개년도)년도 딥러닝 모델 구축

### 0. 라이브러리 및 데이터 불러오기

라이브러리 불러오기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Conv2D, MaxPooling2D, Flatten, Dropout, Reshape
from tensorflow.keras.optimizers import Adam


폰트 지정

In [8]:
import platform
from matplotlib import rc

# 운영 체제에 따라 폰트 설정
if platform.system() == 'Windows':  # Windows 환경
    rc('font', family='Malgun Gothic')  # 맑은 고딕

# 음수 표시 가능하도록 설정
plt.rcParams['axes.unicode_minus'] = False

데이터 불러오기(21 to 22 2개년도 데이터)

In [9]:
RANDOM_STATE = 110

data_21to22 = pd.read_csv("../../project/data/data_21to22.csv", encoding='utf-8-sig', low_memory=False)
print(data_21to22.shape)

(27603, 90)


### 모델링

In [10]:
# 종속 변수는 '2023/부실기업'으로 가정
y = data_21to22['2023/부실기업']
X = data_21to22.drop(columns=['2023/부실기업', '업체코드', '종목명'])

# 데이터 분할 (70:30 비율)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 딥러닝 모형 설정

# 합성곱신경망 (CNN) - 은닉층 5개
cnn_model = Sequential()
cnn_model.add(Reshape((X_train_scaled.shape[1], 1, 1), input_shape=(X_train_scaled.shape[1], 1))) # 입력층
cnn_model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', padding='same')) # 1번째 hidden layer
cnn_model.add(MaxPooling2D(pool_size=(1, 1))) # 2번째 hidden layer
cnn_model.add(Dropout(0.25)) # 3번째 hidden layer
cnn_model.add(Flatten()) # 4번째 hidden layer
cnn_model.add(Dense(128, activation='relu')) # 5번째 hidden layer
cnn_model.add(Dropout(0.25)) # 6번째 hidden layer
cnn_model.add(Dense(1, activation='sigmoid'))  # 출력층
cnn_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# RNN-LSTM - 은닉층 2개
lstm_model = Sequential()
lstm_model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(X_train_scaled.shape[1], 1))) # 1번째 hidden layer
lstm_model.add(LSTM(128, activation='relu')) # 2번째 hidden layer
lstm_model.add(Dense(1, activation='sigmoid')) # 출력층
lstm_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# RNN-GRU - 은닉층 2개
gru_model = Sequential()
gru_model.add(GRU(128, activation='relu', return_sequences=True, input_shape=(X_train_scaled.shape[1], 1))) # 1번째 hidden layer
gru_model.add(GRU(128, activation='relu')) # 2번째 hidden layer
gru_model.add(Dense(1, activation='sigmoid')) # 출력층
gru_model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [12]:
# 딥러닝 모형 학습 및 평가

# CNN 학습 및 평가
X_train_cnn = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1, 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1, 1))
cnn_model.fit(X_train_cnn, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test)[1]
print(f"CNN Accuracy: {cnn_accuracy}")

# RNN-LSTM 학습 및 평가
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))
lstm_model.fit(X_train_lstm, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
lstm_accuracy = lstm_model.evaluate(X_test_lstm, y_test)[1]
print(f"LSTM Accuracy: {lstm_accuracy}")

# RNN-GRU 학습 및 평가
gru_model.fit(X_train_lstm, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
gru_accuracy = gru_model.evaluate(X_test_lstm, y_test)[1]
print(f"GRU Accuracy: {gru_accuracy}")

Epoch 1/200
484/484 [==============================] - 6s 10ms/step - loss: 0.2140 - accuracy: 0.9081 - val_loss: 0.1786 - val_accuracy: 0.9195
Epoch 2/200
484/484 [==============================] - 5s 10ms/step - loss: 0.1723 - accuracy: 0.9239 - val_loss: 0.1946 - val_accuracy: 0.9133
Epoch 3/200
484/484 [==============================] - 5s 10ms/step - loss: 0.1623 - accuracy: 0.9275 - val_loss: 0.1624 - val_accuracy: 0.9247
Epoch 4/200
484/484 [==============================] - 5s 10ms/step - loss: 0.1515 - accuracy: 0.9333 - val_loss: 0.1658 - val_accuracy: 0.9257
Epoch 5/200
484/484 [==============================] - 5s 10ms/step - loss: 0.1449 - accuracy: 0.9351 - val_loss: 0.1560 - val_accuracy: 0.9314
Epoch 6/200
484/484 [==============================] - 5s 10ms/step - loss: 0.1397 - accuracy: 0.9393 - val_loss: 0.1593 - val_accuracy: 0.9288
Epoch 7/200
484/484 [==============================] - 5s 10ms/step - loss: 0.1336 - accuracy: 0.9420 - val_loss: 0.1713 - val_accuracy:

In [13]:
print(f"Deep Learning Models: CNN, LSTM, GRU")
print(f"CNN Accuracy: {cnn_accuracy}")
print(f"LSTM Accuracy: {lstm_accuracy}")
print(f"GRU Accuracy: {gru_accuracy}")

Deep Learning Models: CNN, LSTM, GRU
CNN Accuracy: 0.9326168298721313
LSTM Accuracy: 0.8139113783836365
GRU Accuracy: 0.9340659379959106


.